# 🍔 Restaurant Orders Data Analysis 🍕📊🔮


<h2 style="
    background-color:#cbddca; 
    color:#1e3a8a; 
    padding:15px 20px; 
    border-radius:8px; 
    font-family:sans-serif; 
    font-size:32px; 
    font-weight:bold;
    text-align:center;
    box-shadow: 2px 2px 8px rgba(0,0,0,0.2);
">
📖 Introduction
</h2>


This notebook has been created to transform an ordinary restaurant order log into a compact decision support system that can guide daily operations and menu strategies. The dataset consists of individual order lines, including fields such as Order Number, Customer Name, Food Item, Category, Quantity, Price, Payment Method, and Order Time.

Motivation and Questions

Popular Items 📊 — Which dishes generate the highest quantity and revenue? How concentrated is the sales contribution (e.g., Pareto 80/20 rule)?

Temporal Demand ⏰ — What are the demand patterns across times of day (breakfast/lunch/afternoon/snack/dinner/late night), days of the week, and months?

Payment Behavior 💳 — Are cash, card, or online payments associated with higher ticket sizes?

Basket Associations 🛍️ — Which menu items are frequently purchased together, and could these be bundled into combos?

Customer Value 💎 — Who are my loyal and VIP customers? What is the distribution of spending and frequency?

Profitability Outlook 📈 — By price band, which items are traffic drivers and which are margin drivers? Which products are underperforming?

Customer Segmentation 👨‍👩‍👧‍👦 — Can I segment customers into actionable groups (e.g., one-time customers, frugal regulars, premium VIPs)?

<h2 style="
    background-color:#cbddca; 
    color:#1b3d20; 
    padding:15px 20px; 
    border-radius:8px; 
    font-family:sans-serif; 
    font-size:32px; 
    font-weight:bold;
    text-align:center;
    box-shadow: 2px 2px 8px rgba(0,0,0,0.2);
">
📚 Import Libraries
</h2>


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
# from wordcloud import WordCloud
import os, sys, glob, warnings

<h2 style="
    background-color:#cbddca; 
    color:#1b3d20; 
    padding:15px 20px; 
    border-radius:8px; 
    font-family:sans-serif; 
    font-size:32px; 
    font-weight:bold;
    text-align:center;
    box-shadow: 2px 2px 8px rgba(0,0,0,0.2);
">
📥 Data Loading
</h2>


In [ ]:
df = pd.read_csv("/kaggle/input/restaurant-orders/restaurant_orders.csv")

df.head()


<h2 style="
    background-color:#cbddca; 
    color:#1b3d20; 
    padding:15px 20px; 
    border-radius:8px; 
    font-family:sans-serif; 
    font-size:32px; 
    font-weight:bold;
    text-align:center;
    box-shadow: 2px 2px 8px rgba(0,0,0,0.2);
">
📊 Restaurant Orders Dataset Column
</h2>


This dataset contains records where each row represents an **order**.  
The columns describe the details of each order:  

| 🏷️ Column Name     | 📖 Description |
|--------------------|----------------|
| **Order ID**       | A **unique identifier** for each order. |
| **Customer Name**  | The name of the customer who placed the order. |
| **Food Item**      | The food item ordered by the customer. |
| **Category**       | The category of the food item (e.g., *Main*, *Dessert*). |
| **Quantity**       | The number of items ordered. |
| **Price**          | The **total price** of the order. |
| **Payment Method** | The payment method used by the customer (e.g., *Cash*, *Credit Card*). |
| **Order Time**     | The **date and time** when the order was placed. |



In [ ]:
df.info()

In [ ]:
df.describe().T

<h2 style="
    background-color:#cbddca; 
    color:#1b3d20; 
    padding:15px 20px; 
    border-radius:8px; 
    font-family:sans-serif; 
    font-size:32px; 
    font-weight:bold;
    text-align:center;
    box-shadow: 2px 2px 8px rgba(0,0,0,0.2);
">
🖼️ Data Visualization
</h2>


<h2 style=" color:#2a6040; padding:10px; border-radius:6px; font-family:sans-serif;">
1) Food Item
</h2>

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import random

# Veri setinizden en çok satılan 10 ürünü alalım
# Bu kısım zaten doğru
top_items = df["Food Item"].value_counts().head(10)

# Kendi renk paletiniz
colors = [
    "#eae5c9",
    "#cbddca",
    "#abd6ca",
    "#8ccecb",
    "#6ccc6b",
    "#c0f291",
    "#9bce7d",
    "#75a969",
    "#508554",
    "#2a6040"
]

# Random bir renkte kelimeleri çizecek özel renk fonksiyonu
def random_color_func(word=None, font_size=None, position=None, orientation=None, font_path=None, random_state=None):
    return random.choice(colors)

# WordCloud'u bu özel renk fonksiyonu ile çiz
wordcloud = WordCloud(width=800, height=400, background_color="white", color_func=random_color_func).generate_from_frequencies(top_items)

plt.figure(figsize=(10,6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")  
plt.title("Top 10 Best-Selling Foods", fontsize=16)
plt.show()

Pizza and Brownie stand out as the most popular items. Fries and Soup also show strong sales, while the other foods are sold in smaller amounts. This provides insights into which items should be highlighted on the menu.

<h2 style=" color:#2a6040; padding:10px; border-radius:6px; font-family:sans-serif;">
2) Total Sales by Category
</h2>

In [ ]:
category=df["Category"].value_counts()

colors = [
 
    "#9bce7d",
    "#75a969",
    "#508554",
]

plt.figure(figsize=(8,8))
plt.pie(category,labels=category.index,colors=colors,autopct="%1.1f%%")
plt.title("Total Sales by Category")
plt.show()

This pie chart shows the total sales by category. The sales are very evenly distributed among the three categories: Dessert (33.6%), Main (33.4%), and Starter (33.0%). The distribution is so close that no single category dominates the sales, indicating a balanced performance across the menu.

In [ ]:
df["Order Time"] = pd.to_datetime(df["Order Time"], errors="coerce")
df = df.dropna(subset=["Order Time"]).copy()

df["line_total"] = df["Quantity"] * df["Price"]
df["date"] = df["Order Time"].dt.date
df["year"] = df["Order Time"].dt.year
df["month"] = df["Order Time"].dt.month
df["month_name"] = df["Order Time"].dt.strftime("%b")
df["weekday"] = df["Order Time"].dt.day_name()

df["Order Time"] = pd.to_datetime(df["Order Time"], errors="coerce")
df = df.dropna(subset=["Order Time"]).copy()

df["order_date"] = df["Order Time"].dt.date
df["hour"] = df["Order Time"].dt.floor("h")

def get_daypart(hour):
    if 6 <= hour < 11:   return "Breakfast"
    if 11 <= hour < 15:  return "Lunch"
    if 15 <= hour < 18:  return "Afternoon"
    if 18 <= hour < 22:  return "Dinner"
    return "Late Night"

df["daypart"] = df["Order Time"].dt.hour.apply(get_daypart)

# Output directory
OUTDIR = "./output"
os.makedirs(OUTDIR, exist_ok=True)

# Quick preview
display(df.head())
print(df.shape, "rows, columns:", list(df.columns))


The provided Python code, using the pandas library, has processed a dataset to add several new columns derived from the existing 'Order Time' and 'Quantity' and 'Price' columns. The 'Order Time' column was first converted to a datetime object, and then used to create new columns for the date, year, month, month name, weekday, and hour. Additionally, a new column called line_total was calculated by multiplying the 'Quantity' and 'Price' columns. A custom function, get_daypart, was applied to the 'hour' to categorize each order into one of five dayparts: "Breakfast," "Lunch," "Afternoon," "Dinner," or "Late Night," which was then stored in the new daypart column. This process enriches the dataset, making it more useful for analysis.

<h2 style=" color:#2a6040; padding:10px; border-radius:6px; font-family:sans-serif;">
3) Daily Income
</h2>

In [ ]:
# Haftanın günlerini sıralı hale getirmek için kategori oluşturma
weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
daily_income = df.groupby("weekday")["line_total"].sum().reindex(weekday_order)

# Sadece 7 renk seçimi
colors = [
    "#8ccecb", 
    "#6ccc6b",
    "#c0f291", 
    "#9bce7d",
    "#75a969",
    "#508554",
    "#2a6040"
]

# Grafiği oluşturma
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(daily_income.index, daily_income.values, color=colors, edgecolor="white", linewidth=1.5)

# Başlık ve etiketler
ax.set_title("Total Income by Day of the Week", fontsize=16, weight="bold")
ax.set_xlabel("Week Day", fontsize=12)
ax.set_ylabel("Total Line", fontsize=12)
ax.grid(axis='y', linestyle='--', alpha=0.7)

# Değer etiketlerini ekleme
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., 1.01 * height, 
            f'${int(height):,}', 
            ha='center', va='bottom', fontsize=10, weight="bold")

# Y eksenini sıfırdan başlatmak
ax.set_ylim(bottom=0)

plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

display(daily_income)

This bar chart visualizes the total income by day. Sunday stands out as the highest-earning day with 3,347 in sales. Following a strong start on Monday (2,879), sales dip on Tuesday ($2,443) before steadily increasing and peaking on Thursday (3,096). Sales then decrease on Friday and Saturday, before the significant spike on Sunday. Overall, the weekend, especially Sunday, generates the most income.

<h2 style=" color:#2a6040; padding:10px; border-radius:6px; font-family:sans-serif;">
4) Month Income
</h2>

In [ ]:
# Ayları sıralı hale getirmek için doğru sıralama listesini oluşturun
month_order = [
    "Jan", "Feb", "Mar", "Apr", "May", "Jun", 
    "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"
]

# Aylık geliri doğru sıralama ile gruplandırın
monthly_income = df.groupby("month_name")["line_total"].sum().reindex(month_order)
monthly_income.fillna(0, inplace=True)
# Grafiği oluşturun
plt.figure(figsize=(12, 7))
plt.plot(
    monthly_income.index,
    monthly_income.values,
    marker="o",  
    linestyle="-",
    linewidth=3,
    color="#2a6040" 
)

# Başlıklar ve etiketler
plt.title("Monthly Sales Trend", fontsize=18, weight="bold")
plt.xlabel("Ay", fontsize=14)
plt.ylabel("Total Income ($)", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.6)

# Her bir ay için gelir değerini üzerine yazın
for x, y in zip(monthly_income.index, monthly_income.values):
    plt.text(x, y + 100, f'${int(y):,}', ha="center", va="bottom", fontsize=10, weight="bold")

# Y eksenini sıfırdan başlatın
plt.ylim(bottom=0)

plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

display(monthly_income)

Sales show a general increase from January (2,013) to a peak in March (3,237). After a slight dip in April, sales stabilize until they reach a new, but smaller, peak in July (3,313). A dramatic drop occurs in August ($1,056), followed by sales plummeting to zero for the rest of the year, from September to December.

<h2 style=" color:#2a6040; padding:10px; border-radius:6px; font-family:sans-serif;">
5) Top Customers by Quantity Purchased
</h2>

In [ ]:
customer = df.groupby("Customer Name")["Quantity"].sum()
customer = customer.sort_values(ascending=False).head(20)

plt.figure(figsize=(12, 8))  # Boyutu genişlet
bars = plt.barh(customer.index, customer.values, color="#75a969")
plt.title("Top 20 Customer Orders", fontsize=18)
plt.xlabel("Quantity", fontsize=14)
plt.ylabel("Customer Name", fontsize=14)
plt.gca().invert_yaxis()  # En yüksek değer yukarıda olsun

# Çubuk üzerine değerleri yaz
for bar in bars:
    width = bar.get_width()
    plt.text(width + max(customer.values)*0.01, bar.get_y() + bar.get_height()/2, f"{width:.0f}", 
             va='center', fontsize=12)

plt.tight_layout()
plt.show()


It shows the top 20 customers with the most orders. According to the chart, Jonathan Turner is the customer with the highest number of orders at 8, followed by Michael Smith with 7 orders. Most of the remaining customers have a similar number of orders, around 5 each, indicating an equality in order quantity among these customers.

<h2 style=" color:#2a6040; padding:10px; border-radius:6px; font-family:sans-serif;">
6) Top Customers by Total Order Value
</h2>

In [ ]:
import seaborn as sns

plt.figure(figsize=(12, 8))
sns.barplot(x=customers.values, y=customers.index, palette="Greens_d")
plt.title("Top Customers by Total Order Value", fontsize=18)
plt.xlabel("Total Order Value", fontsize=14)
plt.ylabel("Customer Name", fontsize=14)
plt.show()


The chart displays the customers who spent the most based on total order value. At the top of the chart, Christopher Dunlap, Sara Graham, and Ashley Davis are the highest-spending customers, with an order value of around 120 units. Overall, spending amounts among the top 20 customers are relatively close, but the customers at the top of the chart have slightly higher order values compared to those at the bottom.

<h2 style=" color:#2a6040; padding:10px; border-radius:6px; font-family:sans-serif;">
7) Payment Method
</h2>

In [ ]:
payment=df["Payment Method"].value_counts()

colors = [
    "#eae5c9",
    "#9bce7d",
    "#75a969",
    "#508554",
]

plt.figure(figsize=(8,8))
plt.pie(payment,labels=payment.index,colors=colors,autopct="%1.1f%%")
plt.title("Payment Method")
plt.show()

This pie chart shows the distribution of sales by payment method. The data reveals a very balanced distribution among the four methods, with none of them being significantly more or less preferred than the others. Cash is the most common payment method at 26.4%, followed closely by Credit Card (25.6%), Online Payment (24.2%), and Debit Card (23.8%). The small differences in percentages suggest that customers are comfortable using a variety of payment options.

In [3]:
colors = [
    "#eae5c9",
    "#2a6040",
    "#cbddca",
    "#abd6ca",
    "#508554",
    "#8ccecb",
    "#6ccc6b",
    "#c0f291",
    "#9bce7d",
    "#75a969"  
   
]

ax = orders_pivot.plot(
    kind="bar",
    stacked=True,
    figsize=(12, 7),
    color=colors,  
    edgecolor="white",
    linewidth=1.2
)

plt.title("Distribution of Food Categories by Weekday (%)", fontsize=16, weight="bold")
plt.xlabel("Weekday", fontsize=12)
plt.ylabel("Percentage (%)", fontsize=12)
plt.xticks(rotation=45, ha="right")
plt.legend(title="Food Category", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


NameError: name 'orders_pivot' is not defined

This stacked bar chart shows the percentage distribution of food categories sold each day of the week. The distribution is remarkably consistent across all seven days. The chart indicates that customers have a stable preference for different food categories regardless of the day.

For example, Pasta, Soup, Pizza, and Salad maintain a relatively constant share of sales throughout the week. There is no single food category that experiences a major spike or drop on a particular day, suggesting that demand is not heavily influenced by the weekday. This stability could indicate a predictable and reliable customer base.

<h2 style="
    background-color:#cbddca; 
    color:#1b3d20; 
    padding:15px 20px; 
    border-radius:8px; 
    font-family:sans-serif; 
    font-size:32px; 
    font-weight:bold;
    text-align:center;
    box-shadow: 2px 2px 8px rgba(0,0,0,0.2);
">
🌟 GENEL DEĞERLENDİRME 🌟
</h2>



> [English]
> 
> Based on the provided charts, the business demonstrates a stable and well-balanced performance, with sales distributed fairly evenly across the Main, Dessert, and Starter categories, as well as among different payment methods. This indicates a diverse menu appeal and suggests that no single category or payment type dominates the overall revenue, which is a positive sign of operational balance.Sales follow a clear weekly pattern, with notable peaks on Sunday and Thursday, suggesting that these days attract higher customer traffic or larger order volumes. This pattern could be leveraged for targeted promotions, special offers, or staffing adjustments to optimize service efficiency on peak days. However, a more concerning observation is the significant decline in sales after August, which could be due to seasonal factors, market changes, or operational issues. This trend warrants immediate attention and a thorough investigation to identify potential causes and implement corrective measures.Customer analysis reveals a small group of high-value clients who either place orders frequently or spend significantly more than average. These customers represent a strategic opportunity for targeted marketing campaigns, personalized promotions, and loyalty programs aimed at increasing retention and maximizing lifetime value. Overall, while the general performance of the business is stable, these insights highlight areas for proactive management, strategic planning, and growth-focused interventions.
> 
> 
> [Türkçe]
> 
> Sunulan grafiklere göre, işletme genel olarak istikrarlı ve dengeli bir performans sergilemektedir; satışlar Ana Yemek, Tatlı ve Başlangıç kategorileri ile farklı ödeme yöntemleri arasında oldukça eşit bir şekilde dağılmıştır. Bu durum, menünün farklı müşteri segmentlerine hitap ettiğini ve hiçbir kategori veya ödeme yönteminin gelir üzerinde tek başına baskın olmadığını göstermektedir; bu da operasyonel denge açısından olumlu bir işarettir.Satışlar, özellikle Pazar ve Perşembe günleri zirve yaparak haftalık belirgin bir düzen izlemektedir. Bu, söz konusu günlerde müşteri yoğunluğunun veya sipariş hacminin arttığını göstermektedir. İşletme, bu durumu hedeflenmiş promosyonlar, özel kampanyalar veya personel planlaması gibi stratejilerle değerlendirebilir ve hizmet verimliliğini artırabilir. Ancak, daha dikkat çekici bir gözlem, Ağustos ayından sonra satışlarda yaşanan önemli düşüşlerdir. Bu düşüş mevsimsel faktörler, pazar koşulları veya operasyonel sorunlardan kaynaklanabilir. Bu nedenle, sorunun kaynağını belirlemek ve uygun önlemleri almak için acil bir araştırma yapılması gerekmektedir.Müşteri analizine bakıldığında, sık sık sipariş veren veya yüksek harcama yapan birkaç değerli müşteri öne çıkmaktadır. Bu müşteriler, hedeflenmiş pazarlama kampanyaları, kişiselleştirilmiş promosyonlar ve sadakat programları gibi stratejilerle işletme için önemli fırsatlar yaratmaktadır; hem müşteri bağlılığı artırılabilir hem de yaşam boyu değerleri maksimize edilebilir. Genel olarak işletmenin performansı istikrarlı olsa da, bu analizler proaktif yönetim, stratejik planlama ve büyümeye odaklı müdahaleler için önemli içgörüler sunmaktadır.
